In [4]:
import torch
import torch.nn as nn
import numpy as np
from GPT2 import GPT2classification, GPT2Tokenizer

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''  # 0
device = 'cpu' #'cuda'

def tokenize_input(inputStr, tokenizer, seq_length=1024):
    pad_id = tokenizer.encoder['<pad>']
    tokenized_sentence = tokenizer.encode(inputStr)[:seq_length-20]
    tokens = tokenized_sentence
    token_length = len(tokens)
    tokens.extend([pad_id] * (seq_length - token_length))
    tokens = torch.tensor(tokens, dtype=torch.long)
    return tokens.reshape(1,1024), [token_length]

tokenizer = GPT2Tokenizer(
    'GPT2/bpe/vocab.json',
    'GPT2/bpe/chinese_vocab.model',
    max_len=512)

In [8]:
model = torch.load("../models/financial_finetune_full.pth", 'cpu')
model.eval()
model.to(device)

GPT2classification(
  (GPT2model): GPT2Model(
    (word_embeddings): Embedding(30000, 768)
    (position_embeddings): Embedding(1024, 768)
    (emb_drop): Dropout(p=0.0, inplace=False)
    (transformer): Transformer(
      (layers): ModuleList(
        (0): Block(
          (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attention): Attention(
            (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (resid_drop): Dropout(p=0.0, inplace=False)
            (dense): Linear(in_features=768, out_features=768, bias=True)
          )
          (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
            (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          )
        )
        (1): Block(
          (inp

In [27]:
inputStr = '这句话里有"金融"，因此分类结果应该是financial'  # the text you want to classify

tokens, token_length = tokenize_input(inputStr, tokenizer, seq_length=1024)
output = model(tokens, token_length)
output = torch.softmax(output, dim=1)

print('financial probability:', int(output[0][1].item()*100), '%')

financial probability: 91 %
